# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [93]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [94]:
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = state
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')
    
    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return sum(state[1])

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [95]:
from queue import PriorityQueue

def astar(problem: Problem):
    visited = set()
    queue = PriorityQueue()
    queue.put((problem.heuristic(problem.initial_state), (), problem.initial_state))
    while not queue.empty():
        current = queue.get()
        if current[1] not in visited:
            path = list(current[1])
            visited.add(current[2])
            if problem.is_goal(current[2]):
                print(len(visited))
                return path
            actions = problem.available_actions(current[2])
            for action in actions:
                new_state = problem.do_action(current[2], action)
                if new_state not in visited:
                    queue.put((len(path) + problem.heuristic(new_state) + problem.action_cost(current[2], action), tuple(path+[action]), new_state))

Now lets test your code in the vacuum world!

In [96]:
astar(VacuumProblem())

6


['Suck', 'Right', 'Suck']

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [97]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return len(state[1]) - sum(state[1])
    
astar(VacuumProblem1())

7


['Suck', 'Right', 'Suck']

And another in which heuristic grossly overestimates the cost.

In [98]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state):
        return 10 * sum(state[1])
    
astar(VacuumProblem2())

4


['Suck', 'Right', 'Suck']

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

The best heuristic function is VacuumProblem2, bacause it visits less states than the other ones. It is not expected answer, because this heuristic overestimates the cost. In this example we are lucky, because despite this heuristic grossly overestimates the cost our astar algorithm returns correct answer. When we have overestimate heuristic our astar algorithm will visit less states before returning result, but we don't have any guarantee that this result is correct. Also when we have underestimate heuristic astar will always return correct answer, but for sure it will visit more states.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [99]:
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        return ((1,1), (7, 2, 4), (5, 0, 6), (8, 3, 1))
        
    def available_actions(self, state):
        actions = ["up", "left", "right", "down"]
        if state[0][0] == 0:
          actions.remove("up")
        elif state[0][0] == 2:
          actions.remove("down")
        if state[0][1] == 0:
          actions.remove("left")
        elif state[0][1] == 2:
          actions.remove("right")
        return actions
        
    def do_action(self, state, action):
        new_state = list(list(tup) for tup in state)
        y, x = new_state[0]
        if action == "up":
          new_state[y+1][x] = new_state[y][x]
          new_state[y][x] = 0
          new_state[0][0]-=1
        elif action == "down":
          new_state[y+1][x] = new_state[y+2][x]
          new_state[y+2][x] = 0
          new_state[0][0]+=1
        elif action == "left":
          new_state[y+1][x] = new_state[y+1][x-1]
          new_state[y+1][x-1] = 0
          new_state[0][1]-=1
        elif action == "right":
          new_state[y+1][x] = new_state[y+1][x+1]
          new_state[y+1][x+1] = 0
          new_state[0][1]+=1
        return tuple(tuple(lis) for lis in new_state)
    
    def is_goal(self, state) -> bool:
        return state == ((0,0), (0, 1, 2), (3, 4, 5), (6, 7, 8))
        
    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state) -> float:
        return 0

**Prove that this heuristic is admissible.**

heuristic 0 is admissible, bacause we assume, that we don't have to do any action to reach the goal.

In [100]:
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state) -> float:     
        # Propose another heuristic here
        cnt = 0
        for i in range(1,4):
            row = list(state[i])
            for j in range(len(row)):
                if (i-1)*3+j != row[j]:
                    cnt+=1
        return max(cnt-1, 0)

**Prove that this heuristic is admissible.**

When we move puzzle to blank space we can reduce puzzles that are on wrong places by maximum 2, but even when we set puzzle and blank space on right places and it is not a goal state in next move we have to move something to blank space and automaticly blank space will not be on the right place. Only when there is one move to reach goal, we can reduce our puzzles that are on wrong places by 2. Imagine situation when we have to do our last move when puzzle with "1" and blank space are on wrong places our heuristic will return 2, but in fact we have to do only 1 action. That's why we have to substract one from our value, and select maximum from this value and 0, because heuristic can not be negative.

In [101]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state) -> float:
        # Propose yet another heuristic here
        cnt = 0
        for i in range(1,4):
            row = list(state[i])
            for j in range(len(row)):
                cnt += abs(row[j]%3 - j) + abs(row[j]//3 - (i-1))
        return cnt/2

**Prove that this heuristic is admissible.**

Manhattan distance gives us distance which we have to go through to get puzzle on right place. When we sum this distances for each puzzle we get number of moves that is needed to reach the goal. But since by exchanging puzzles we can set 2 of then at once on the right places we have to divide our distance by 2. Then we know that our distance in not overestimated.

Run your heuristics on the given problem.

In [102]:
plan0 = astar(PuzzleProblem())
plan1 = astar(PuzzleProblem1())
plan2 = astar(PuzzleProblem2())

print("Is plan0==plan1?", plan0 == plan1)
print("Is plan0==plan2?", plan0 == plan2)
print("Is plan1==plan2?", plan1 == plan2)

170249
39397
28287
Is plan0==plan1? True
Is plan0==plan2? True
Is plan1==plan2? True


**Which of the heuristics is the best for this task? Why is that?**

*Replace this text with an answer*

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.